### [Code](https://github.com/ksm26/AI-Agentic-Design-Patterns-with-AutoGen/blob/main/L2_Sequential_Chats_and_Customer_Onboarding.ipynb)
### [Video](https://www.bilibili.com/video/BV1WJ4m137dk?spm_id_from=333.788.videopod.episodes&vd_source=1d3a7b81d826789081d8b6870d4fff8e&p=3)

一个假设的人机交互场景
1. 收集客户信息
2. 调查客户兴趣
3. 基于收集信息与客户交流

![attachment:image.png](https://github.com/ksm26/AI-Agentic-Design-Patterns-with-AutoGen/blob/main/images/l2.png?raw=true)

![l3.png](https://github.com/ksm26/AI-Agentic-Design-Patterns-with-AutoGen/blob/main/images/l3.png?raw=true)

In [1]:
import sys
sys.path.append('/opt/project/KG_Assist_LLM')

In [2]:
config_list = [
  {
    "model": "qwen2.5:72b-instruct-q4_0",
    "base_url": "http://192.168.28.5:11434/v1",
    "api_key": "ollama",
    "price": [0, 0], 
  }
]
llm_config={"config_list": config_list}

In [3]:
from autogen import ConversableAgent

### Creating the needed agents

创建一个收集用用户信息的agent

In [4]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

创建一个收集用户喜好的agent

In [5]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

创建一个agent, 主要在于收集完用户基本信息后, 根据信息来提供有趣的故事,笑话, 相当于最后的交互agent

In [6]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    # is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

创建一个agent, 这个agent的human_input_mode设置为了always, 用于从 用户那里获取输入

In [7]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

### Creating tasks
Now, you can craft a series of tasks to facilitate the onboarding process.

这里定义了一个按顺序对话的系统, max_turns用于控制每个agent的对话轮数
- summary_prompt: 我们通常希望把上一个轮的对话结果保留下来, 以便于给下一个大模型使用, summary_prompt用于将模型对话内容进行总结摘要.传递给下一个大模型
- 

In [8]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

### Start the onboarding process
Note: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.


In [9]:
from autogen import initiate_chats
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/root/miniconda3/envs/langchain/lib/python3.11/site-packages/autogen/agentchat/chat.py:53: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customer_proxy_agent (to Onboarding Personal Information Agent):

yangfan

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Hi Yangfan, nice to meet you! Could you also share where you're located, please?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

shanghai

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{"name": "Yangfan", "location": "Shanghai"}

-------------------------------------------------------------------------------

### Print out the summary

依次输出每次系统总结的内容

In [10]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{"name": "Yangfan", "location": "Shanghai"}


The takeaway is that Yangfan, located in Shanghai, expressed interest in anime when asked about topics they enjoy reading.


The takeaway is that Yangfan, located in Shanghai, expressed interest in anime and was then provided with interesting facts about the cultural connections between anime and Shanghai, including local history, inspiration for popular anime, and community events.


